In [44]:
import fitsio
import balltracking.balltrack as blt
import os, glob
import fitsio
from scipy.ndimage.filters import gaussian_filter 
from scipy.signal import convolve2d
from scipy.io import readsav
import numpy as np
import matplotlib
matplotlib.use('agg')
from matplotlib import cm
import matplotlib.pyplot as plt
# Load FLCT
from pathlib import Path
import time
import numpy as np

matplotlib.rcParams.update({'font.size': 14})
matplotlib.rcParams.update({'xtick.labelsize': 12, 'ytick.labelsize': 12})

In [70]:
lon_lats = [(60.4, 0), (60.4, 60), (60.4, 70), (0, 0), (350, 0)]
nevents = len(lon_lats)
basenames = ['mtrack_20110627_200034_TAI_20110628_000033_TAI_Postel_{:05.1f}_{:04.1f}_magnetogram.fits'
                 .format(lon_lat[0], lon_lat[1]) for lon_lat in lon_lats]
# Get the intensity files
magfiles = [os.path.join(os.environ['DATA'], 'SDO/HMI/polar_study/', basename) for basename in basenames]
lanesdirs = [os.path.join(os.environ['DATA'], 'SDO/HMI/polar_study/lonCR_{:05.1f}_lat_{:04.1f}'.format(lon_lat[0], lon_lat[1]))
                for lon_lat in lon_lats]


In [71]:
fwhm = 11
nframes = 320
tavgs = range(40,nframes+1,40)
tranges = [[0, tavg] for tavg in tavgs]
print(tranges)
nflows = len(tranges)

[[0, 40], [0, 80], [0, 120], [0, 160], [0, 200], [0, 240], [0, 280], [0, 320]]


In [84]:
x = np.arange(512) * 368/1000
y = x
for idx, lanesdir in enumerate(lanesdirs[-3:]):
#     idx = 0
#     lanesdir = lanesdirs[idx]
    print('processing lanesdir: ', lanesdir)
    magfile = magfiles[idx]
    # Get all the lanes files at 11 px
    lanes_files = sorted(glob.glob(os.path.join(lanesdir, 'lanes_fwhm{:d}*.fits'.format(fwhm))))
    # Average of unsigned flux at increasing time averages
    mag = fitsio.read(magfile)

    for i,trange in enumerate(tranges):
    # i = 7
    # trange = tranges[i]
        tavg_min = int(np.round(trange[1]*45/60))
        print(tavg_min)
        lanes = fitsio.read(lanes_files[i])

        mag_abs = np.abs(mag)
        mag_abs_avg = mag_abs[trange[0]:trange[1], ...].mean(axis=0)


        plt.figure(figsize=(10,10))
        plt.imshow(lanes, cmap='gray_r', origin='lower', extent=(0, x[-1], 0, y[-1]))
        plt.contourf(mag_abs_avg, np.linspace(30,1000,10), colors ='red', extent=(0, x[-1], 0, y[-1]))
        plt.xlabel('X [Mm]', fontsize=12)
        plt.ylabel('Y [Mm]', fontsize=12)
        plt.title('FWHM = 4 Mm   Time average = {:d} min'.format(tavg_min), fontsize=12)
        plt.tight_layout()
        figf = os.path.join(lanesdir, 'lanes_mag_{:d}.png'.format(i))
        plt.savefig(figf, dpi=300)
        print(figf)
        plt.close()

processing lanesdir:  /Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0
30
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_0.png
60
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_1.png
90
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_2.png
120
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_3.png
150
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_4.png
180
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_5.png
210
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_6.png
240
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_060.4_lat_70.0/lanes_mag_7.png
processing lanesdir:  /Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0
30
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_mag_0.png
60
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_mag_1.png
90
/Users/rattie/Data/SDO/HMI/polar_study

In [85]:
# Without magnetic flux

for idx, lanesdir in enumerate(lanesdirs[-2:]):
#     idx = 0
#     lanesdir = lanesdirs[idx]
    print('processing lanesdir: ', lanesdir)
    # Get all the lanes files at 11 px
    lanes_files = sorted(glob.glob(os.path.join(lanesdir, 'lanes_fwhm{:d}*.fits'.format(fwhm))))
    # Average of unsigned flux at increasing time averages
    for i,trange in enumerate(tranges):
    # i = 7
    # trange = tranges[i]
        tavg_min = int(np.round(trange[1]*45/60))
        lanes = fitsio.read(lanes_files[i])

        plt.figure(figsize=(10,10))
        plt.imshow(lanes, cmap='gray_r', origin='lower', extent=(0, x[-1], 0, y[-1]))
        plt.xlabel('X [Mm]', fontsize=12)
        plt.ylabel('Y [Mm]', fontsize=12)
        plt.title('FWHM = 4 Mm   Time average = {:d} min'.format(tavg_min), fontsize=12)
        plt.tight_layout()
        figf = os.path.join(lanesdir, 'lanes_{:d}.png'.format(i))
        plt.savefig(figf, dpi=300)
        print(figf)
        plt.close()

processing lanesdir:  /Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_0.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_1.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_2.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_3.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_4.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_5.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_6.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_000.0_lat_00.0/lanes_7.png
processing lanesdir:  /Users/rattie/Data/SDO/HMI/polar_study/lonCR_350.0_lat_00.0
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_350.0_lat_00.0/lanes_0.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_350.0_lat_00.0/lanes_1.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR_350.0_lat_00.0/lanes_2.png
/Users/rattie/Data/SDO/HMI/polar_study/lonCR